# Predicting Appointment No Shows  with Amazon SageMaker Linear Learner
_**Supervised Learning with Logistic Regression: A Binary Prediction Problem**_

---

---

## Contents

1. [Background](#Background)
1. [Preparation](#Preparation)
1. [Data](#Data)
    1. [Exploration](#Exploration)
    1. [Transformation](#Transformation)
1. [Training](#Training)
1. [Hosting](#Hosting)
1. [Evaluation](#Evaluation)
1. [Extensions](#Extensions)

---

## Background


This notebook uses the linear learner algorithm to predict whether a patient will be a no-show for a medical appointment.The dataset was downloaded from Kaggle https://www.kaggle.com/joniarroba/noshowappointments. The appointment data of 110,527 records was collected from medical clinics in the city of Vitoria, Brazil, over a three month period in 2016.

The following steps were undertaken:

* Preparing your Amazon SageMaker notebook
* Downloading data from the internet into Amazon SageMaker
* Investigating and transforming the data so that it can be fed to Amazon SageMaker algorithms
* Estimating a model using the Gradient Boosting algorithm
* Evaluating the effectiveness of the model
* Setting the model up to make on-going predictions

---

## Preparation

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

## Notes
Tried a simplified model with only a few features to see if that any impact on the recall. It didn't improve.

In [1]:
bucket = 'sagemaker-sf-strategenics'
prefix = 'sagemaker/DEMO-xgboost-noShow'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Bring in the Python libraries that we'll use throughout the analysis

In [2]:
from datetime import date
from datetime import time
from datetime import datetime
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
from sagemaker.predictor import csv_serializer    # Converts strings for HTTP POST requests on inference
import seaborn as sns

---

## Data
The csv file containing the data is stored in an S3 bucket. First let's read the data file into a Pandas data frame.

In [3]:
data_key = 'appointmentData.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

noShow= pd.read_csv(data_location)

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
noShow
noShow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


Let's start by renaming some of the columns

In [4]:
noShow.rename(columns = {'Hipertension': 'Hypertension',
                         'Handcap': 'Disabilities'}, inplace = True)

print(noShow.columns)

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hypertension',
       'Diabetes', 'Alcoholism', 'Disabilities', 'SMS_received', 'No-show'],
      dtype='object')


As we have the date that the appointment was scheduled and the date of the appointment, we can calculate the number of days that the patient waited for the appointment.

First we have to convert the two date columns to a date format.

In [5]:
#convert date columns to a date format
noShow['tempSchedDate'] = pd.to_datetime(noShow['ScheduledDay'])
noShow['tempAppDate'] = pd.to_datetime(noShow['AppointmentDay'])
#get the date part of the date columns, as the Scheduled date has a time component but the appointment day does not
noShow['AppointmentDate']= noShow['tempAppDate'].dt.date
noShow['AppointmentBooked']= noShow['tempSchedDate'].dt.date
#calculate the waiting time
noShow['WaitingTime'] = (noShow.AppointmentDate - noShow.AppointmentBooked).dt.days

print(noShow.AppointmentBooked.head())
print(noShow.AppointmentDate.head())
print(noShow.WaitingTime.head())
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page


0    2016-04-29
1    2016-04-29
2    2016-04-29
3    2016-04-29
4    2016-04-29
Name: AppointmentBooked, dtype: object
0    2016-04-29
1    2016-04-29
2    2016-04-29
3    2016-04-29
4    2016-04-29
Name: AppointmentDate, dtype: object
0    0
1    0
2    0
3    0
4    0
Name: WaitingTime, dtype: int64


Calculate the day of the week for the appointment day and drop the original scheduled and appointment columns.

In [6]:
#Find the day of the week of the appointment
noShow['DayOfWeek'] = noShow['tempAppDate'].dt.day_name()

#drop the columns no longer needed
noShow = noShow.drop(['ScheduledDay','AppointmentDay','tempSchedDate','tempAppDate'], axis=1)

print(noShow.AppointmentDate.head())
print(noShow.DayOfWeek.head())
print('Day of week:', sorted(noShow.DayOfWeek.unique()))

0    2016-04-29
1    2016-04-29
2    2016-04-29
3    2016-04-29
4    2016-04-29
Name: AppointmentDate, dtype: object
0    Friday
1    Friday
2    Friday
3    Friday
4    Friday
Name: DayOfWeek, dtype: object
Day of week: ['Friday', 'Monday', 'Saturday', 'Thursday', 'Tuesday', 'Wednesday']


    There are no duplicate Appointment IDs so we will index the data by AppointmentID. We also need to convert PatientId to an integer.

In [7]:
noShow.PatientId = noShow.PatientId.astype('int64')
print(noShow.PatientId.head())
noShow.set_index('AppointmentID', inplace = True)

0     29872499824296
1    558997776694438
2      4262962299951
3       867951213174
4      8841186448183
Name: PatientId, dtype: int64


In [ ]:
noShow
noShow.info()


# Data Summary

* There are 110,527 patient records, and 13 features for each patient
* The features are mixed; some numeric, some categorical


*Features:*
* `Age`: Patient's age. Integer -1 to 115
* `Gender`: Patient's gender, string M,F
* `Alcoholism`: Binary, 1=yes
* `Diabetes`:Binary, 1= yes
* `Hypertension`:Binary, 1= yes
* `Disabilities`: The number of disabilities for a patient. Integer, 1-4
* `Scholarship`: This indicates whether the patient receives financial support from the government. Binary, 1=yes
* `Neighbourhood`: This is the location of the medical clinic. String, 80 values
* `SMS_receceived`:Whether they received a SMS reminder before the appointment. Binary, 1= yes
* `AppointmentBooked`: Date that the appointment was booked
* `AppointmentDate`: Date of the appointment
* `DayOfWeek`: The weekday of the appointment. Integer, 0-5
* `WaitingTime`: The number of days between booking the appointment and the appointment date. Integer, -6 to 179


*Target variable:*
* `No-show`: Was the patient a no-show? Binary: 1=yes,0=no
Overall, 20% of the patients were no-shows.

In [8]:
noShow['Age'] = np.where(noShow['Age']<0, np.nan, noShow['Age'])
noShow['Age'].describe()
noShow['WaitingTime'] = np.where(noShow['WaitingTime']<0, np.nan, noShow['WaitingTime'])
noShow['WaitingTime'].describe()

count    110522.000000
mean         10.184253
std          15.255115
min           0.000000
25%           0.000000
50%           4.000000
75%          15.000000
max         179.000000
Name: WaitingTime, dtype: float64

Disability has values from 0-5, indicating the number of disabilities a patient has. We will turn this into a binary column to indicate whether the patient has a disablity or not.

In [9]:
noShow['Disability'] = np.where(noShow['Disabilities']>1, 1, noShow['Disabilities'])
print('Disability:', sorted(noShow.Disability.unique()))
count = noShow.groupby(['Disability', 'Disabilities']).size() 
print(count)  


Disability: [0, 1]
Disability  Disabilities
0           0               108286
1           1                 2042
            2                  183
            3                   13
            4                    3
dtype: int64


In [ ]:
Now we will look at patient history as it is possible that people who have a previous no-show are more likely to no-show again.

In [10]:
#determine if a patient has had a previous appointment
noShow.sort_values(by=['PatientId','AppointmentDate'], inplace=True)
pd.options.display.max_rows=100
noShow['PreviousAppointment'] = noShow.sort_values(by = ['PatientId','AppointmentDate']).groupby(['PatientId']).cumcount()
#print(noShow[['PatientId','AppointmentDate', 'PreviousAppointment']].head(100)) 


a = noShow.groupby(pd.cut(noShow.PreviousAppointment, bins = [-1, 0,1,2,3,4,5, 85], include_lowest = True))[['PreviousAppointment']].count()
b = pd.DataFrame(a)
b.set_index(pd.Series(['0', '1', '2', '3', '4', '5', '> 5']))

,PreviousAppointment
0,62299
1,24379
2,10484
3,4984
4,2617
5,1498
> 5,4264


In [ ]:
Calculate whether the patient has a previous no-show

In [11]:
noShow['NoShow']=np.where(noShow['No-show'] == "Yes", 1,0)
count = noShow.groupby(['No-show', 'NoShow']).size() 
print(count)

#noShow['PreviousNoShow'] = (noShow[noShow['PreviousAppointment'] > 0].sort_values(['PatientId', 'AppointmentDate']).groupby(['PatientId'])['NoShow'].cumsum())
noShow['NumberOfPreviousNoShow'] = (noShow.sort_values(['PatientId', 'AppointmentDate']).groupby(['PatientId'])['NoShow'].cumsum())
noShow['PreviousNoShows']=noShow['NumberOfPreviousNoShow']-noShow['NoShow']
noShow['PreviousNoShowProp'] = noShow['PreviousNoShows']/ noShow[noShow['PreviousAppointment'] > 0]['PreviousAppointment']

pd.options.display.max_rows=100
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
print(noShow[['PatientId','AppointmentDate', 'No-show', 'PreviousAppointment','PreviousNoShows','PreviousNoShowProp']].head(100)) 

noShow['PreviousNoShowProp'].describe()

No-show  NoShow
No       0         88208
Yes      1         22319
dtype: int64
               PatientId AppointmentDate No-show  PreviousAppointment  PreviousNoShows  PreviousNoShowProp
AppointmentID                                                                                             
5751990            39217      2016-06-03      No                    0                0                 NaN
5760144            43741      2016-06-01      No                    0                0                 NaN
5712759            93779      2016-05-18      No                    0                0                 NaN
5637648           141724      2016-05-02      No                    0                0                 NaN
5637728           537615      2016-05-06      No                    0                0                 NaN
5680449          5628261      2016-05-13     Yes                    0                0                 NaN
5718578         11831856      2016-05-19      No                 

count    48228.000000
mean         0.198413
std          0.343029
min          0.000000
25%          0.000000
50%          0.000000
75%          0.333333
max          1.000000
Name: PreviousNoShowProp, dtype: float64

In [ ]:
#drop columns that we no longer need
noShow = noShow.drop(['NoShow','AppointmentBooked','PreviousNoShows'], axis=1)
noShow.info()

In [ ]:
noShow.info()

Change age and waiting time to categorical columns


In [12]:
def WaitingTimeCat(days):
    if days == 0:
        return '0 days'
    elif  days in range(1,8):
        return '1-7 days'
    elif  days in range(8,15):
        return '8-14 days'
    elif days in range(15, 29):
        return '15-28 days'
    else:
        return '> 28 days'
    
def AgeCat(years):
    if years in range(0,5):
        return '0-4 years'
    elif  years in range(5,15):
        return '05-14 years'
    elif  years in range(15,25):
        return '15-24 years'
    elif years in range(25, 35):
        return '25-34 years'
    elif years in range(35, 45):
        return '35-44 years'
    elif years in range(45, 55):
        return '45-54 years'
    elif years in range(55, 65):
        return '55-64 years'
    else:
        return '> 64 years'   
noShow['WaitingTimeCat'] = noShow.WaitingTime.apply(WaitingTimeCat)
noShow['AgeCat'] = noShow.Age.apply(AgeCat)



In [13]:
noShow.describe()

,PatientId,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Disabilities,SMS_received,WaitingTime,Disability,PreviousAppointment,NoShow,NumberOfPreviousNoShow,PreviousNoShows,PreviousNoShowProp
count,1.105270e+05,110526.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110522.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,48228.000000
mean,1.474963e+14,37.089219,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026,10.184253,0.020276,1.270314,0.201933,0.406561,0.204629,0.198413
std,2.560949e+14,23.110026,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873,15.255115,0.140942,3.913419,0.401444,0.797339,0.647026,0.343029
min,3.921700e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,15.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.333333
max,9.999816e+14,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,179.000000,1.000000,87.000000,1.000000,18.000000,17.000000,1.000000


In [14]:
display(pd.crosstab(noShow.Age,noShow.AgeCat))

AgeCat,0-4 years,05-14 years,15-24 years,25-34 years,35-44 years,45-54 years,55-64 years,> 64 years
Age,,,,,,,,
0.0,3539,0,0,0,0,0,0,0
1.0,2273,0,0,0,0,0,0,0
2.0,1618,0,0,0,0,0,0,0
3.0,1513,0,0,0,0,0,0,0
4.0,1299,0,0,0,0,0,0,0
5.0,0,1489,0,0,0,0,0,0
6.0,0,1521,0,0,0,0,0,0
7.0,0,1427,0,0,0,0,0,0
8.0,0,1424,0,0,0,0,0,0


In [26]:
#drop PatientID, AppointmentID and Neightbourhood - check whether we can drop and index column
#noShow = noShow.drop(['PreviousNoShowProp'], axis=1)
noShow['NoShow']=np.where(noShow['No-show'] == "Yes", 1,0)
df = noShow[['NoShow', 'Gender', 'AgeCat','WaitingTimeCat','NumberOfPreviousNoShow']].copy()
df2=df.dropna()
df2.describe()
print(df2.shape)
#model_data = df[['NoShow', 'Gender', 'AgeCat','WaitingTimeCat','NumberOfPreviousNoShow']].copy()

#create indicator columns for categorical columns, as requried by XGBoost
model_data = pd.get_dummies(df2) 
model_data.head()
model_data.info()

(110527, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 110527 entries, 5751990 to 5660958
Data columns (total 17 columns):
NoShow                       110527 non-null int64
NumberOfPreviousNoShow       110527 non-null int64
Gender_F                     110527 non-null uint8
Gender_M                     110527 non-null uint8
AgeCat_0-4 years             110527 non-null uint8
AgeCat_05-14 years           110527 non-null uint8
AgeCat_15-24 years           110527 non-null uint8
AgeCat_25-34 years           110527 non-null uint8
AgeCat_35-44 years           110527 non-null uint8
AgeCat_45-54 years           110527 non-null uint8
AgeCat_55-64 years           110527 non-null uint8
AgeCat_> 64 years            110527 non-null uint8
WaitingTimeCat_0 days        110527 non-null uint8
WaitingTimeCat_1-7 days      110527 non-null uint8
WaitingTimeCat_15-28 days    110527 non-null uint8
WaitingTimeCat_8-14 days     110527 non-null uint8
WaitingTimeCat_> 28 days     110527 non-null uint8
dty

Split the dataset into training data (70%), validation data (20%) and prediction data (10%)

In [19]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=8147), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%


In [20]:
print(train_data.shape)
print(validation_data.shape)
print(test_data.shape)


(77368, 17)
(22106, 17)
(11053, 17)


Amazon SageMaker's Linear Learner container expects data in the libSVM or CSV data format.  For this example, we'll stick to CSV.  Note that the first column must be the target variable and the CSV should not include headers.  Also, notice that although repetitive it's easiest to do this after the train|validation|test split rather than before.  This avoids any misalignment issues due to random reordering.

In [21]:
pd.concat([train_data['NoShow'], train_data.drop(['NoShow'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['NoShow'], validation_data.drop(['NoShow'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

Now we'll copy the file to S3 for Amazon SageMaker's managed training to pickup.

In [22]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---

## Training
Now we know that most of our features have skewed distributions, some are highly correlated with one another, and some appear to have non-linear relationships with our target variable.  Also, for targeting future prospects, good predictive accuracy is preferred to being able to explain why that prospect was targeted.  Taken together, these aspects make gradient boosted trees a good candidate algorithm.

There are several intricacies to understanding the algorithm, but at a high level, gradient boosted trees works by combining predictions from many simple models, each of which tries to address the weaknesses of the previous models.  By doing this the collection of simple models can actually outperform large, complex models.  Other Amazon SageMaker notebooks elaborate on gradient boosting trees further and how they differ from similar algorithms.

`xgboost` is an extremely popular, open-source package for gradient boosted trees.  It is computationally powerful, fully featured, and has been successfully used in many machine learning competitions.  Let's start with a simple `xgboost` model, trained using Amazon SageMaker's managed, distributed training framework.

First we'll need to specify the ECR container location for Amazon SageMaker's implementation of XGBoost.

In [23]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3, which also specify that the content type is CSV.

In [24]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='text/csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='text/csv')

In [ ]:
#train_file = 'linear_train.data'

#f = io.BytesIO()
#smac.write_numpy_to_dense_tensor(f, train_X.astype('float32'), train_y.astype('float32'))
#f.seek(0)

#boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', train_file)).upload_fileobj(f)

First we'll need to specify training parameters to the estimator.  This includes:
1. The `Linear Learner` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [25]:


sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=16,
                           predictor_type='binary_classifier',
                           mini_batch_size=200)

linear.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

2019-11-10 23:10:04 Starting - Starting the training job...
2019-11-10 23:10:06 Starting - Launching requested ML instances......
2019-11-10 23:11:06 Starting - Preparing the instances for training......
2019-11-10 23:12:15 Downloading - Downloading input data...
2019-11-10 23:12:55 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[11/10/2019 23:12:58 INFO 139718227969856] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2864224859840512, "sum": 0.2864224859840512, "min": 0.2864224859840512}}, "EndTime": 1573427605.871246, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1573427605.871186}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2802113945620048, "sum": 0.2802113945620048, "min": 0.2802113945620048}}, "EndTime": 1573427605.871321, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1573427605.871308}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2873173018934813, "sum": 0.2873173018934813, "min": 0.2873173018934813}}, "EndTime": 1573427605.871358, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learne

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27618126108238733, "sum": 0.27618126108238733, "min": 0.27618126108238733}}, "EndTime": 1573427620.619009, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1573427620.618954}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2722138442894338, "sum": 0.2722138442894338, "min": 0.2722138442894338}}, "EndTime": 1573427620.619087, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1573427620.619074}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2765828875309445, "sum": 0.2765828875309445, "min": 0.2765828875309445}}, "EndTime": 1573427620.61913, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Lear

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27309316185471927, "sum": 0.27309316185471927, "min": 0.27309316185471927}}, "EndTime": 1573427635.339784, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1573427635.339725}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27097221596871013, "sum": 0.27097221596871013, "min": 0.27097221596871013}}, "EndTime": 1573427635.339861, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1573427635.339848}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27328252411876935, "sum": 0.27328252411876935, "min": 0.27328252411876935}}, "EndTime": 1573427635.339901, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Line

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27196449240254617, "sum": 0.27196449240254617, "min": 0.27196449240254617}}, "EndTime": 1573427650.175878, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1573427650.175824}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2708360841237202, "sum": 0.2708360841237202, "min": 0.2708360841237202}}, "EndTime": 1573427650.175952, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1573427650.17594}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27209860742400965, "sum": 0.27209860742400965, "min": 0.27209860742400965}}, "EndTime": 1573427650.175996, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear L

...

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27064363944098124, "sum": 0.27064363944098124, "min": 0.27064363944098124}}, "EndTime": 1573427753.422302, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1573427753.422238}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27070566992685585, "sum": 0.27070566992685585, "min": 0.27070566992685585}}, "EndTime": 1573427753.42238, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1573427753.422367}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27068699367305776, "sum": 0.27068699367305776, "min": 0.27068699367305776}}, "EndTime": 1573427753.422434, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Lin

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2706246357754722, "sum": 0.2706246357754722, "min": 0.2706246357754722}}, "EndTime": 1573427768.097574, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1573427768.097515}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27069609387550947, "sum": 0.27069609387550947, "min": 0.27069609387550947}}, "EndTime": 1573427768.097649, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1573427768.097636}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27064871032003296, "sum": 0.27064871032003296, "min": 0.27064871032003296}}, "EndTime": 1573427768.097706, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linea

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27060786484436666, "sum": 0.27060786484436666, "min": 0.27060786484436666}}, "EndTime": 1573427782.705227, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1573427782.70516}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2706857842361371, "sum": 0.2706857842361371, "min": 0.2706857842361371}}, "EndTime": 1573427782.705306, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1573427782.705292}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27062086589595813, "sum": 0.27062086589595813, "min": 0.27062086589595813}}, "EndTime": 1573427782.705349, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2705964988253895, "sum": 0.2705964988253895, "min": 0.2705964988253895}}, "EndTime": 1573427797.40088, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1573427797.400812}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.2706741129672589, "sum": 0.2706741129672589, "min": 0.2706741129672589}}, "EndTime": 1573427797.400958, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1573427797.400945}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.27058519738943465, "sum": 0.27058519738943465, "min": 0.27058519738943465}}, "EndTime": 1573427797.401013, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Le

[11/10/2019 23:16:43 INFO 139718227969856] #early_stopping_criteria_metric: host=algo-1, epoch=14, criteria=sampled_accuracy, value=0.920655025785
[11/10/2019 23:16:43 INFO 139718227969856] Epoch 14: Loss improved. Updating best model
[11/10/2019 23:16:43 INFO 139718227969856] #validation_score (algo-1) : ('binary_classification_cross_entropy_objective', 0.49658531893715757)
[11/10/2019 23:16:43 INFO 139718227969856] #validation_score (algo-1) : ('binary_classification_accuracy', 0.9207454989595585)
[11/10/2019 23:16:43 INFO 139718227969856] #validation_score (algo-1) : ('binary_f_1.000', 0.8231017770597738)
[11/10/2019 23:16:43 INFO 139718227969856] #validation_score (algo-1) : ('precision', 0.7368040491684743)
[11/10/2019 23:16:43 INFO 139718227969856] #validation_score (algo-1) : ('recall', 0.9322964318389753)
[11/10/2019 23:16:43 INFO 139718227969856] #quality_metric: host=algo-1, validation binary_classification_cross_entropy_objective <loss>=0.496585318937
[11/10/2019 23:16:43 IN

In [ ]:
#!conda install -y -c conda-forge xgboost 

In [ ]:
#don't run. Need to change to work with linear learner          
import tarfile
import pickle as pkl
import xgboost

# download the model artifact from AWS S3
!aws s3 cp s3://sagemaker-sf-strategenics/sagemaker/DEMO-xgboost-noShow/output/sagemaker-xgboost-2019-11-08-00-23-47-910/output/model.tar.gz .

#opens the downloaded model artifcat and loads it as 'model' variable
tar = tarfile.open('model.tar.gz')
tar.extractall()
tar.close()
model = pkl.load(open('xgboost-model', 'rb'))            

In [ ]:
#don't run. Need to change to work with linear learner 
#map_names = dict(zip(model.feature_names, train_data[cols_input].columns))
#model.feature_names = list(map_names.values())

#plot feature importance
fig, ax = plt.subplots(figsize=(12,12))
xgboost.plot_importance(model, importance_type='gain', max_num_features=30, height=0.8, ax=ax, show_values = False)
plt.title('Feature Importance')
plt.show()

In [ ]:
train_data.info()

---

## Hosting
Now that we've trained the `Linear Learner` algorithm on our data, let's deploy a model that's hosted behind a real-time endpoint.

In [27]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

-------------------------------------------------------------------------------------------------!

---

## Evaluation
There are many ways to compare the performance of a machine learning model, but let's start by simply comparing actual to predicted values.  In this case, we're simply predicting whether the customer subscribed to a term deposit (`1`) or not (`0`), which produces a simple confusion matrix.

First we'll need to determine how we pass data into and receive data from our endpoint.  Our data is currently stored as NumPy arrays in memory of our notebook instance.  To send it in an HTTP POST request, we'll serialize it as a CSV string and then decode the resulting CSV.

*Note: For inference with CSV format, SageMaker XGBoost requires that the data does NOT include the target variable.*

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batches to CSV string payloads (notice, we drop the target variable from our dataset first)
1. Retrieve mini-batch predictions by invoking the Linear Learner endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
test_data.describe()

In [35]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer


test_features = test_data.drop(['NoShow'], axis=1)
test_features=test_features.iloc[0]
test_features
test_df = np.array(test_features.values).astype('float32')

#prediction = linear_predictor.predict(tf)
    
for tf in test_features:
    prediction = linear_predictor.predict(tf)
    print(prediction)

ValueError: ('Unable to handle input format: ', <class 'int'>)

Now we'll check our confusion matrix to see how well we predicted versus actuals.

In [ ]:
result = linear_predictor.predict(train_data[0][30:31])
print(result)
### Predict on Test Data
#preds_test_lin = batch_predict_linear(test_data.iloc[:,1:], 100, linear_predictor , 'text/csv')

In [ ]:
pd.crosstab(index=test_data['NoShow'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'],margins=True)

## In summary, of the 11053 patients, we predicted 97 would be a no-show and 2,277 of them actually didn't turn up to the appointment.  We also had 2,180 that were no-shows but we predicted as turning up.  This is less than desirable.  


### (Optional) Clean-up

If you are done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
sagemaker.Session().delete_endpoint(linear_predictor.endpoint)